<a href="https://colab.research.google.com/github/Kolawole-a2/Kola_Projects/blob/main/AFOLABI_CyberAnalytics_Tools_SEAS8414_HW1_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1: Install required packages
!pip install pandas numpy transformers --quiet

# Step 2: Use an interactive uploader for notebooks (e.g., Google Colab or Jupyter)
from IPython.display import display
from google.colab import files
import pandas as pd
import numpy as np

# Upload file
uploaded = files.upload()

# Assuming only one file is uploaded
filename = list(uploaded.keys())[0]

# Step 3: Load and explore the data
df = pd.read_csv(filename)

# Show basic info
print("DataFrame loaded successfully!")
print("\nShape of DataFrame:", df.shape)
print("\nColumns:", df.columns.tolist())
print("\nFirst 5 rows:")
display(df.head())


Saving Cyberattacks Detection.csv to Cyberattacks Detection (5).csv
DataFrame loaded successfully!

Shape of DataFrame: (100000, 16)

Columns: ['Attack ID', 'Source IP', 'Destination IP', 'Source Country', 'Destination Country', 'Protocol', 'Source Port', 'Destination Port', 'Attack Type', 'Payload Size (bytes)', 'Detection Label', 'Confidence Score', 'ML Model', 'Affected System', 'Port Type', 'Timestamp']

First 5 rows:


<ipython-input-1-1493761289>:17: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)


,Attack ID,Source IP,Destination IP,Source Country,Destination Country,Protocol,Source Port,Destination Port,Attack Type,Payload Size (bytes),Detection Label,Confidence Score,ML Model,Affected System,Port Type,Timestamp
0,1.0,68.156.227.194,194.91.121.112,India,USA,ICMP,16824.0,53410.0,Brute Force,3936.0,Detected,0.600507,K-Nearest Neighbors,Cloud Storage,Other,NaN
1,2.0,249.221.229.41,3.207.120.235,Germany,USA,TCP,23977.0,31745.0,Advanced Persistent Threat (APT),2870.0,Detected,0.524150,K-Nearest Neighbors,Network Router,Other,NaN
2,4.0,154.55.209.171,132.115.33.82,USA,China,UDP,36556.0,38880.0,Cross-Site Scripting (XSS) Attack,3833.0,Not Detected,0.238944,Neural Network,Workstation,Other,NaN
3,5.0,106.61.204.27,40.61.49.130,France,France,UDP,14724.0,21669.0,Insider Threat,4301.0,Not Detected,0.122334,Neural Network,Workstation,Other,NaN
4,6.0,144.190.250.125,34.68.66.192,South Korea,Germany,TCP,44693.0,6474.0,Watering Hole Attack,4569.0,Detected,0.970895,Neural Network,Application Server,Other,NaN


In [ ]:
df.columns.tolist()



['Attack ID',
 'Source IP',
 'Destination IP',
 'Source Country',
 'Destination Country',
 'Protocol',
 'Source Port',
 'Destination Port',
 'Attack Type',
 'Payload Size (bytes)',
 'Detection Label',
 'Confidence Score',
 'ML Model',
 'Affected System',
 'Port Type',
 'Timestamp']

In [ ]:
# Filter for brute force attacks
brute_force_df = df[df['Attack Type'].str.contains('brute force', case=False, na=False)]

# Group by source and destination country and count occurrences
country_pairs = (
    brute_force_df
    .groupby(['Source Country', 'Destination Country'])
    .size()
    .reset_index(name='Attack Count')
)

# Find the pair with the highest attack count
most_common_pair = country_pairs.loc[country_pairs['Attack Count'].idxmax()]

print("Most common source-destination country pair for brute force attacks:")
print(f"Source Country: {most_common_pair['Source Country']}")
print(f"Destination Country: {most_common_pair['Destination Country']}")
print(f"Attack Count: {most_common_pair['Attack Count']}")


Most common source-destination country pair for brute force attacks:
Source Country: Japan
Destination Country: France
Attack Count: 68


In [ ]:
# Define a threshold for high payload size (e.g., 75th percentile)
payload_threshold = df['Payload Size (bytes)'].quantile(0.75)

# Filter rows where Detection Label is 'Not Detected' (adjust if label differs)
filtered_df = df[
    (df['Detection Label'].str.lower() == 'not detected') &
    (df['Payload Size (bytes)'] > payload_threshold)
]

# Group by 'Attack Type' and calculate average confidence score
avg_confidence = (
    filtered_df
    .groupby('Attack Type')['Confidence Score']
    .mean()
    .reset_index()
    .sort_values(by='Confidence Score', ascending=False)
)

# Get the attack type with the highest average confidence
top_attack = avg_confidence.iloc[0]

print("Attack Type with Highest Average Confidence (Not Detected, High Payload):")
print(f"Attack Type: {top_attack['Attack Type']}")
print(f"Average Confidence Score: {top_attack['Confidence Score']:.4f}")


Attack Type with Highest Average Confidence (Not Detected, High Payload):
Attack Type: File Transfer Protocol (FTP) Attack
Average Confidence Score: 0.2534


In [ ]:
# Filter for detected attacks
detected_df = df[df['Detection Label'].str.lower() == 'detected']

# Further filter for APT attacks (case-insensitive contains 'advanced persistent threat' or 'apt')
apt_df = detected_df[
    detected_df['Attack Type'].str.contains('advanced persistent threat|apt', case=False, na=False)
]

# Further filter for 'Workstation' in 'Affected System'
workstation_apt_df = apt_df[
    apt_df['Affected System'].str.contains('workstation', case=False, na=False)
]

# Group by 'Protocol' and calculate median confidence score
median_confidence_by_protocol = (
    workstation_apt_df
    .groupby('Protocol')['Confidence Score']
    .median()
    .reset_index()
    .sort_values(by='Confidence Score', ascending=True)
)

# Get the protocol with the lowest median confidence score
lowest_median_protocol = median_confidence_by_protocol.iloc[0]

print("Protocol with the lowest median confidence score for detected APT attacks on Workstations:")
print(f"Protocol: {lowest_median_protocol['Protocol']}")
print(f"Median Confidence Score: {lowest_median_protocol['Confidence Score']:.4f}")


Protocol with the lowest median confidence score for detected APT attacks on Workstations:
Protocol: ICMP
Median Confidence Score: 0.7085


In [ ]:
# Extract month name safely using .loc to avoid SettingWithCopyWarning
filtered_df = filtered_df.copy()  # Make an explicit copy first to be safe
filtered_df.loc[:, 'Month'] = filtered_df['Timestamp'].dt.month_name()


In [ ]:
import pandas as pd
import calendar

# Ensure 'Timestamp' is datetime
df['Timestamp'] = pd.to_datetime(df['Timestamp'], errors='coerce')

# Filter for 2024 only
df_2024 = df[df['Timestamp'].dt.year == 2024]

# Filter for undetected Zero-Day Exploit attacks on Application Server
filtered_df = df_2024[
    (df_2024['Detection Label'].str.lower().isin(['not detected', 'undetected'])) &
    (df_2024['Attack Type'].str.contains('zero-day exploit', case=False, na=False)) &
    (df_2024['Affected System'].str.contains('application server', case=False, na=False))
]

# Make an explicit copy to avoid SettingWithCopyWarning
filtered_df = filtered_df.copy()

# Add 'Month' column safely
filtered_df.loc[:, 'Month'] = filtered_df['Timestamp'].dt.month_name()

# Count attacks by month
monthly_counts = (
    filtered_df
    .groupby('Month')
    .size()
    .reset_index(name='Attack Count')
)

# Sort months by calendar order (Jan to Dec)
monthly_counts['Month'] = pd.Categorical(monthly_counts['Month'], categories=calendar.month_name[1:], ordered=True)
monthly_counts = monthly_counts.sort_values('Month')

# Find month with highest attacks
top_month = monthly_counts.loc[monthly_counts['Attack Count'].idxmax()]

print(f"Month in 2024 when Application Server systems had the highest number of undetected Zero-Day Exploit attacks:")
print(f"{top_month['Month']} with {top_month['Attack Count']} attacks")


Month in 2024 when Application Server systems had the highest number of undetected Zero-Day Exploit attacks:
September with 5 attacks


In [ ]:
import pandas as pd
import numpy as np

# Filter for undetected Drive-by Download attacks on Workstation systems
filtered_df = df[
    (df['Detection Label'].str.lower().isin(['not detected', 'undetected'])) &
    (df['Attack Type'].str.contains('drive-by download', case=False, na=False)) &
    (df['Affected System'].str.contains('workstation', case=False, na=False))
].copy()

# Define port ranges and labels
port_bins = [10000, 20000, 30000, 40000, 50000]
port_labels = ['10000-20000', '20000-30000', '30000-40000', '40000-50000']

# Filter rows where Destination Port is within 10000 to 50000 (inclusive)
filtered_df = filtered_df[
    (filtered_df['Destination Port'] >= 10000) & (filtered_df['Destination Port'] <= 50000)
]

# Create a new column 'Port Range' using pd.cut
filtered_df['Port Range'] = pd.cut(
    filtered_df['Destination Port'],
    bins=port_bins,
    labels=port_labels,
    right=True,
    include_lowest=True
)

# Group by port range and calculate average confidence score
avg_confidence_by_port_range = (
    filtered_df
    .groupby('Port Range')['Confidence Score']
    .mean()
    .reset_index()
    .sort_values(by='Confidence Score', ascending=False)
)

# Get the port range with the highest average confidence score
top_port_range = avg_confidence_by_port_range.iloc[0]

print("Destination port range with the highest average confidence score for undetected Drive-by Download attacks on Workstations:")
print(f"Port Range: {top_port_range['Port Range']}")
print(f"Average Confidence Score: {top_port_range['Confidence Score']:.4f}")


Destination port range with the highest average confidence score for undetected Drive-by Download attacks on Workstations:
Port Range: 40000-50000
Average Confidence Score: 0.2773


<ipython-input-11-3240015609>:32: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby('Port Range')['Confidence Score']


In [ ]:
# Filter for detected Cross-Site Scripting attacks on Workstations with confidence > 0.8
filtered_df = df[
    (df['Detection Label'].str.lower() == 'detected') &
    (df['Confidence Score'] > 0.8) &
    (df['Attack Type'].str.contains('cross-site scripting|xss', case=False, na=False)) &
    (df['Affected System'].str.contains('workstation', case=False, na=False))
]

# Limit to the specified ML models
models_of_interest = [
    'Random Forest',
    'K-Nearest Neighbors',
    'Neural Network',
    'Logistic Regression'
]

filtered_df = filtered_df[filtered_df['ML Model'].isin(models_of_interest)]

# Count detections per ML model
model_counts = (
    filtered_df
    .groupby('ML Model')
    .size()
    .reset_index(name='Detection Count')
    .sort_values(by='Detection Count', ascending=False)
)

# Get the model with the highest detection count
top_model = model_counts.iloc[0]

print("Machine learning model that detected the most Cross-Site Scripting (XSS) attacks on Workstations with confidence > 0.8:")
print(f"Model: {top_model['ML Model']}")
print(f"Number of detections: {top_model['Detection Count']}")


Machine learning model that detected the most Cross-Site Scripting (XSS) attacks on Workstations with confidence > 0.8:
Model: K-Nearest Neighbors
Number of detections: 26


In [ ]:
# Filter for undetected Insider Threat attacks using UDP with Source Port > 50000
filtered_df = df[
    (df['Detection Label'].str.lower().isin(['not detected', 'undetected'])) &
    (df['Attack Type'].str.contains('insider threat', case=False, na=False)) &
    (df['Protocol'].str.upper() == 'UDP') &
    (df['Source Port'] > 50000)
]

# Group by destination country and count attacks
country_counts = (
    filtered_df
    .groupby('Destination Country')
    .size()
    .reset_index(name='Attack Count')
    .sort_values(by='Attack Count', ascending=False)
)

# Get the destination country with the most attacks
top_country = country_counts.iloc[0]

print("Destination country paired with UDP protocol having most undetected Insider Threat attacks with Source Port > 50000:")
print(f"Destination Country: {top_country['Destination Country']}")
print(f"Number of attacks: {top_country['Attack Count']}")


Destination country paired with UDP protocol having most undetected Insider Threat attacks with Source Port > 50000:
Destination Country: USA
Number of attacks: 27


In [ ]:
import pandas as pd

# Ensure Timestamp is datetime
df['Timestamp'] = pd.to_datetime(df['Timestamp'], errors='coerce')

# Filter for Q2 2024: April (4), May (5), June (6)
df_q2_2024 = df[
    (df['Timestamp'].dt.year == 2024) &
    (df['Timestamp'].dt.month.isin([4, 5, 6]))
]

# Filter for detected Ping of Death attacks on Database Server systems
filtered_df = df_q2_2024[
    (df['Detection Label'].str.lower() == 'detected') &
    (df['Attack Type'].str.contains('ping of death', case=False, na=False)) &
    (df['Affected System'].str.contains('database server', case=False, na=False))
]

# Group by ML Model and count attacks
model_counts = (
    filtered_df
    .groupby('ML Model')
    .size()
    .reset_index(name='Detection Count')
    .sort_values(by='Detection Count', ascending=False)
)

# Get the ML model with highest number of detected attacks
top_model = model_counts.iloc[0]

print("Machine learning model with the highest number of detected 'Ping of Death' attacks targeting Database Server systems in Q2 2024:")
print(f"Model: {top_model['ML Model']}")
print(f"Number of detections: {top_model['Detection Count']}")


Machine learning model with the highest number of detected 'Ping of Death' attacks targeting Database Server systems in Q2 2024:
Model: Neural Network
Number of detections: 2


<ipython-input-14-772898915>:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_df = df_q2_2024[


In [ ]:
# Filter for detected Phishing attacks on Email Server systems
filtered_df = df[
    (df['Detection Label'].str.lower() == 'detected') &
    (df['Attack Type'].str.contains('phishing', case=False, na=False)) &
    (df['Affected System'].str.contains('email server', case=False, na=False))
]

# Group by ML Model and calculate variance of Confidence Score
variance_df = (
    filtered_df
    .groupby('ML Model')['Confidence Score']
    .var()
    .reset_index()
    .rename(columns={'Confidence Score': 'Confidence Variance'})
    .sort_values('Confidence Variance')
)

# Get the model with the lowest variance
lowest_variance_model = variance_df.iloc[0]

print("Machine learning model with the lowest variance in confidence scores for detected Phishing attacks targeting Email Server systems:")
print(f"Model: {lowest_variance_model['ML Model']}")
print(f"Confidence Score Variance: {lowest_variance_model['Confidence Variance']:.6f}")


Machine learning model with the lowest variance in confidence scores for detected Phishing attacks targeting Email Server systems:
Model: Random Forest
Confidence Score Variance: 0.018521


In [ ]:
import pandas as pd

# Ensure Timestamp is datetime
df['Timestamp'] = pd.to_datetime(df['Timestamp'], errors='coerce')

# Filter for 2024 detected Brute Force attacks on Web Server with confidence > 0.9
filtered_df = df[
    (df['Timestamp'].dt.year == 2024) &
    (df['Detection Label'].str.lower() == 'detected') &
    (df['Attack Type'].str.contains('brute force', case=False, na=False)) &
    (df['Affected System'].str.contains('web server', case=False, na=False)) &
    (df['Confidence Score'] > 0.9)
]

# Group by Source Country and count attacks
country_counts = (
    filtered_df
    .groupby('Source Country')
    .size()
    .reset_index(name='Attack Count')
    .sort_values(by='Attack Count', ascending=False)
)

# Get source country with the highest count
top_country = country_counts.iloc[0]

print("Source country with the highest number of detected Brute Force attacks on Web Server systems in 2024 (confidence > 0.9):")
print(f"Source Country: {top_country['Source Country']}")
print(f"Number of attacks: {top_country['Attack Count']}")


Source country with the highest number of detected Brute Force attacks on Web Server systems in 2024 (confidence > 0.9):
Source Country: Brazil
Number of attacks: 1


In [ ]:
# Filter for detected Brute Force attacks on Web Server systems in 2024 with confidence > 0.9
filtered_df = df[
    (df['Timestamp'].dt.year == 2024) &
    (df['Detection Label'].str.lower() == 'detected') &
    (df['Attack Type'].str.contains('brute force', case=False, na=False)) &
    (df['Affected System'].str.contains('web server', case=False, na=False)) &
    (df['Confidence Score'] > 0.9) &
    (df['Source Country'].isin(['China', 'Japan', 'France', 'USA']))
]

# Group by Source Country and count attacks
country_counts = (
    filtered_df
    .groupby('Source Country')
    .size()
    .reset_index(name='Attack Count')
    .sort_values(by='Attack Count', ascending=False)
)

# Get the top source country
top_country = country_counts.iloc[0]

print("In 2024, source country with the highest number of detected Brute Force attacks on Web Servers with confidence > 0.9:")
print(f"Country: {top_country['Source Country']}")
print(f"Number of attacks: {top_country['Attack Count']}")


In 2024, source country with the highest number of detected Brute Force attacks on Web Servers with confidence > 0.9:
Country: China
Number of attacks: 1


In [ ]:
# Filter for not detected SQL Injection attacks on Database Server systems
filtered_df = df[
    (df['Detection Label'].str.lower().isin(['not detected', 'undetected'])) &
    (df['Attack Type'].str.contains('sql injection', case=False, na=False)) &
    (df['Affected System'].str.contains('database server', case=False, na=False))
]

# Group by Protocol and calculate average Payload Size
avg_payload = (
    filtered_df
    .groupby('Protocol')['Payload Size (bytes)']
    .mean()
    .reset_index()
    .sort_values(by='Payload Size (bytes)', ascending=False)
)

# Get the protocol with highest average payload size
top_protocol = avg_payload.iloc[0]

print("Protocol with the highest average payload size for not detected SQL Injection attacks on Database Server systems:")
print(f"Protocol: {top_protocol['Protocol']}")
print(f"Average Payload Size (bytes): {top_protocol['Payload Size (bytes)']:.2f}")


Protocol with the highest average payload size for not detected SQL Injection attacks on Database Server systems:
Protocol: ICMP
Average Payload Size (bytes): 2638.99


In [ ]:
import pandas as pd
import numpy as np

# Ensure Timestamp is datetime
df['Timestamp'] = pd.to_datetime(df['Timestamp'], errors='coerce')

# Filter for detected DDoS attacks on Firewall systems in 2024
filtered_df = df[
    (df['Timestamp'].dt.year == 2024) &
    (df['Detection Label'].str.lower() == 'detected') &
    (df['Attack Type'].str.contains('distributed denial of service|ddos', case=False, na=False)) &
    (df['Affected System'].str.contains('firewall', case=False, na=False))
].copy()

# Calculate median payload size across these filtered attacks
median_payload = filtered_df['Payload Size (bytes)'].median()

# Keep only attacks with payload size above median
filtered_df = filtered_df[filtered_df['Payload Size (bytes)'] > median_payload]

# Define a function to map month to quarter label
def month_to_quarter(month):
    if month in [1, 2, 3]:
        return 'Q1 (Jan-Mar)'
    elif month in [4, 5, 6]:
        return 'Q2 (Apr-Jun)'
    elif month in [7, 8, 9]:
        return 'Q3 (Jul-Sep)'
    else:
        return 'Q4 (Oct-Dec)'

# Create a 'Quarter' column
filtered_df['Quarter'] = filtered_df['Timestamp'].dt.month.apply(month_to_quarter)

# Group by Quarter and count attacks
quarter_counts = (
    filtered_df
    .groupby('Quarter')
    .size()
    .reset_index(name='Attack Count')
    .sort_values(by='Attack Count', ascending=False)
)

# Get the quarter with the highest number of attacks
top_quarter = quarter_counts.iloc[0]

print("Quarter in 2024 when Firewall systems faced the most detected DDoS attacks with payload size above median:")
print(f"{top_quarter['Quarter']} with {top_quarter['Attack Count']} attacks")


Quarter in 2024 when Firewall systems faced the most detected DDoS attacks with payload size above median:
Q2 (Apr-Jun) with 4 attacks


In [ ]:
# Filter for detected FTP Attack instances in 2024 with confidence > 0.7
filtered_df = df[
    (df['Timestamp'].dt.year == 2024) &
    (df['Detection Label'].str.lower() == 'detected') &
    (df['Attack Type'].str.contains('file transfer protocol|ftp attack', case=False, na=False)) &
    (df['Confidence Score'] > 0.7)
]

# Group by Source Country and count
country_counts = (
    filtered_df
    .groupby('Source Country')
    .size()
    .reset_index(name='Attack Count')
    .sort_values(by='Attack Count', ascending=False)
)

# Get the source country with highest count
top_country = country_counts.iloc[0]

print("Source country with the highest number of detected FTP Attack instances with confidence > 0.7 in 2024:")
print(f"Country: {top_country['Source Country']}")
print(f"Number of attacks: {top_country['Attack Count']}")


Source country with the highest number of detected FTP Attack instances with confidence > 0.7 in 2024:
Country: Brazil
Number of attacks: 25


In [ ]:
# Step 1: Filter dataset for Email Server attacks in 2024 and the specified attack types
target_attacks = ['Credential Stuffing', 'SQL Injection', 'Brute Force', 'Phishing']

filtered_df = df[
    (df['Timestamp'].dt.year == 2024) &
    (df['Affected System'].str.contains('email server', case=False, na=False)) &
    (df['Attack Type'].str.lower().isin([a.lower() for a in target_attacks]))
].copy()

# Step 2: Compute the 25th percentile of payload sizes across the filtered data
q1_payload = filtered_df['Payload Size (bytes)'].quantile(0.25)

# Step 3: Create a new column for identifying payloads below Q1
filtered_df['Below_Q1'] = filtered_df['Payload Size (bytes)'] < q1_payload

# Step 4: Compute proportion of attacks below Q1 for each attack type
proportion_df = (
    filtered_df
    .groupby('Attack Type')
    .agg(
        Total_Attacks=('Below_Q1', 'count'),
        Below_Q1_Count=('Below_Q1', 'sum')
    )
    .assign(Proportion=lambda x: x['Below_Q1_Count'] / x['Total_Attacks'])
    .sort_values('Proportion', ascending=False)
    .reset_index()
)

# Step 5: Get attack type with highest proportion
top_attack = proportion_df.iloc[0]

print("Attack type targeting Email Server systems in 2024 with the highest proportion of payloads below 25th percentile:")
print(f"Attack Type: {top_attack['Attack Type']}")
print(f"Proportion Below Q1: {top_attack['Proportion']:.2%}")


Attack type targeting Email Server systems in 2024 with the highest proportion of payloads below 25th percentile:
Attack Type: Brute Force
Proportion Below Q1: 31.34%


In [ ]:
# Step 1: Filter for undetected attacks on IoT Device systems in 2024
filtered_df = df[
    (df['Timestamp'].dt.year == 2024) &
    (df['Detection Label'].str.lower().isin(['not detected', 'undetected'])) &
    (df['Affected System'].str.contains('iot device', case=False, na=False))
].copy()

# Step 2: Group by Attack Type and calculate median confidence score
median_confidence = (
    filtered_df
    .groupby('Attack Type')['Confidence Score']
    .median()
    .reset_index()
    .sort_values(by='Confidence Score', ascending=False)
)

# Step 3: Get the attack type with the highest median confidence
top_attack = median_confidence.iloc[0]

print("Attack type targeting IoT Device systems with highest median confidence score for undetected attacks in 2024:")
print(f"Attack Type: {top_attack['Attack Type']}")
print(f"Median Confidence Score: {top_attack['Confidence Score']:.2f}")


Attack type targeting IoT Device systems with highest median confidence score for undetected attacks in 2024:
Attack Type: Ping of Death
Median Confidence Score: 0.33


In [ ]:
# Step 1: Filter for detected attacks targeting Network Router systems
filtered_df = df[
    (df['Detection Label'].str.lower() == 'detected') &
    (df['Affected System'].str.contains('network router', case=False, na=False))
].copy()

# Step 2: Group by Attack Type and calculate standard deviation of payload sizes
std_payloads = (
    filtered_df
    .groupby('Attack Type')['Payload Size (bytes)']
    .std()
    .reset_index()
    .sort_values(by='Payload Size (bytes)', ascending=False)
)

# Step 3: Get the attack type with the highest standard deviation
top_attack = std_payloads.iloc[0]

print("Attack type with highest standard deviation in payload sizes for detected attacks on Network Router systems:")
print(f"Attack Type: {top_attack['Attack Type']}")
print(f"Standard Deviation of Payload Size: {top_attack['Payload Size (bytes)']:.2f}")


Attack type with highest standard deviation in payload sizes for detected attacks on Network Router systems:
Attack Type: Phishing
Standard Deviation of Payload Size: 1518.37


In [ ]:
# Step 1: Filter for undetected attacks targeting Cloud Storage with confidence < 0.3
filtered_df = df[
    (df['Detection Label'].str.lower().isin(['not detected', 'undetected'])) &
    (df['Affected System'].str.contains('cloud storage', case=False, na=False)) &
    (df['Confidence Score'] < 0.3)
].copy()

# Step 2: Group by Attack Type and calculate average payload size
avg_payloads = (
    filtered_df
    .groupby('Attack Type')['Payload Size (bytes)']
    .mean()
    .reset_index()
    .sort_values(by='Payload Size (bytes)', ascending=False)
)

# Step 3: Get the attack type with the highest average payload size
top_attack = avg_payloads.iloc[0]

print("Attack type with highest average payload size for undetected attacks targeting Cloud Storage (confidence < 0.3):")
print(f"Attack Type: {top_attack['Attack Type']}")
print(f"Average Payload Size: {top_attack['Payload Size (bytes)']:.2f} bytes")


Attack type with highest average payload size for undetected attacks targeting Cloud Storage (confidence < 0.3):
Attack Type: Ping of Death
Average Payload Size: 2696.43 bytes


In [ ]:
# Step 1: Filter data based on conditions
filtered_df = df[
    (df['Attack Type'].str.contains('cryptojacking', case=False, na=False)) &
    (df['Source Country'].str.lower() == 'india') &
    (df['Source Port'] < 20000)
].copy()

# Step 2: Add a binary column for detection status
filtered_df['Detected'] = filtered_df['Detection Label'].str.lower() == 'detected'

# Step 3: Group by Destination Country and calculate detection rate
detection_rates = (
    filtered_df
    .groupby('Destination Country')['Detected']
    .agg(['sum', 'count'])
    .rename(columns={'sum': 'Detected Count', 'count': 'Total'})
    .assign(Detection_Rate=lambda x: x['Detected Count'] / x['Total'])
    .sort_values(by='Detection_Rate', ascending=False)
    .reset_index()
)

# Step 4: Get the country with the highest detection rate
top_country = detection_rates.iloc[0]

print("Destination country with the highest detection rate for Cryptojacking attacks from India with source ports < 20000:")
print(f"Country: {top_country['Destination Country']}")
print(f"Detection Rate: {top_country['Detection_Rate']:.2%}")
print(f"Detected Attacks: {top_country['Detected Count']} / {top_country['Total']}")


Destination country with the highest detection rate for Cryptojacking attacks from India with source ports < 20000:
Country: Brazil
Detection Rate: 77.78%
Detected Attacks: 14 / 18


In [ ]:
# List of ML models to check
ml_models = ['K-Nearest Neighbors', 'Support Vector Machine', 'Neural Network', 'Logistic Regression']

# Filter for SNMP attacks on Cloud Storage systems
filtered_df = df[
    (df['Attack Type'].str.contains('simple network management protocol|snmp attack', case=False, na=False)) &
    (df['Affected System'].str.contains('cloud storage', case=False, na=False)) &
    (df['ML Model'].isin(ml_models))
].copy()

# Group by ML Model to calculate detection rate
detection_rates = (
    filtered_df
    .groupby('ML Model')
    .agg(
        total_attacks=('Detection Label', 'count'),
        detected_attacks=('Detection Label', lambda x: (x.str.lower() == 'detected').sum())
    )
    .assign(detection_rate=lambda x: x['detected_attacks'] / x['total_attacks'])
    .reset_index()
    .sort_values(by='detection_rate', ascending=False)
)

# Get model with highest detection rate
top_model = detection_rates.iloc[0]

print("Machine learning model with the highest detection rate for SNMP attacks on Cloud Storage systems:")
print(f"Model: {top_model['ML Model']}")
print(f"Detection Rate: {top_model['detection_rate']:.2%}")


Machine learning model with the highest detection rate for SNMP attacks on Cloud Storage systems:
Model: Logistic Regression
Detection Rate: 60.00%


In [ ]:
# Step 1: Filter SNMP attacks on Cloud Storage systems
snmp_df = df[
    (df['Attack Type'].str.contains('simple network management protocol|snmp attack', case=False, na=False)) &
    (df['Affected System'].str.contains('cloud storage', case=False, na=False))
].copy()

# Step 2: Calculate detection rate by ML model
# Group by ML Model and Detection Label counts
model_detection_counts = (
    snmp_df
    .groupby(['ML Model', 'Detection Label'])
    .size()
    .unstack(fill_value=0)
)

# Ensure consistent column names (e.g., lowercase)
model_detection_counts.columns = [col.lower() for col in model_detection_counts.columns]

# Calculate detection rate per model
# 'detected' count / total attacks by that model
model_detection_counts['total'] = model_detection_counts.sum(axis=1)
model_detection_counts['detection_rate'] = model_detection_counts.get('detected', 0) / model_detection_counts['total']

# Find the model with the highest detection rate
top_model = model_detection_counts['detection_rate'].idxmax()
top_rate = model_detection_counts.loc[top_model, 'detection_rate']

print("Machine learning model with the highest detection rate for SNMP attacks on Cloud Storage systems:")
print(f"Model: {top_model}")
print(f"Detection Rate: {top_rate:.2%}")


Machine learning model with the highest detection rate for SNMP attacks on Cloud Storage systems:
Model: Logistic Regression
Detection Rate: 60.00%


In [ ]:
# Step 1: Filter for detected MITM attacks on Network Router systems using TCP protocol
filtered_df = df[
    (df['Detection Label'].str.lower() == 'detected') &
    (df['Attack Type'].str.contains('man-in-the-middle|mitm', case=False, na=False)) &
    (df['Affected System'].str.contains('network router', case=False, na=False)) &
    (df['Protocol'].str.upper() == 'TCP')
].copy()

# Step 2: Group by Source Country and count attacks
country_counts = (
    filtered_df
    .groupby('Source Country')
    .size()
    .reset_index(name='Attack Count')
    .sort_values(by='Attack Count', ascending=False)
)

# Step 3: Get source country with the highest count
top_country = country_counts.iloc[0]

print("Source country paired with TCP protocol that had the most detected MITM attacks on Network Router systems:")
print(f"Country: {top_country['Source Country']}")
print(f"Number of Attacks: {top_country['Attack Count']}")


Source country paired with TCP protocol that had the most detected MITM attacks on Network Router systems:
Country: USA
Number of Attacks: 13


In [ ]:
# List of countries to check
countries_of_interest = ['Russia', 'South Korea', 'Germany', 'UK']

# Filter the dataframe for detected MITM attacks on Network Router with TCP protocol and countries of interest
filtered_df = df[
    (df['Detection Label'].str.lower() == 'detected') &
    (df['Attack Type'].str.contains('man-in-the-middle|mitm', case=False, na=False)) &
    (df['Affected System'].str.contains('network router', case=False, na=False)) &
    (df['Protocol'].str.upper() == 'TCP') &
    (df['Source Country'].isin(countries_of_interest))
].copy()

# Count attacks per source country
country_counts = (
    filtered_df
    .groupby('Source Country')
    .size()
    .reset_index(name='Attack Count')
    .sort_values(by='Attack Count', ascending=False)
)

print("Detected MITM attacks on Network Router systems by source country (TCP protocol):")
print(country_counts)

# Get top country
if not country_counts.empty:
    top_country = country_counts.iloc[0]
    print(f"\nTop source country: {top_country['Source Country']} with {top_country['Attack Count']} attacks")
else:
    print("No matching data found for the specified criteria.")


Detected MITM attacks on Network Router systems by source country (TCP protocol):
  Source Country  Attack Count
0        Germany            13
1         Russia            12
2    South Korea            11
3             UK            10

Top source country: Germany with 13 attacks


In [ ]:
# Step 1: Filter Email Spam attacks targeting Firewall systems with payload > 4000 bytes
email_spam_df = df[
    (df['Attack Type'].str.contains('email spam', case=False, na=False)) &
    (df['Affected System'].str.contains('firewall', case=False, na=False)) &
    (df['Payload Size (bytes)'] > 4000)
].copy()

# Step 2: Group by Source Country and Detection Label counts
country_detection_counts = (
    email_spam_df
    .groupby(['Source Country', 'Detection Label'])
    .size()
    .unstack(fill_value=0)
)

# Normalize column names to lowercase for consistency
country_detection_counts.columns = [col.lower() for col in country_detection_counts.columns]

# Step 3: Calculate detection rate per source country
country_detection_counts['total'] = country_detection_counts.sum(axis=1)
country_detection_counts['detection_rate'] = country_detection_counts.get('detected', 0) / country_detection_counts['total']

# Step 4: Find the source country with highest detection rate
top_country = country_detection_counts['detection_rate'].idxmax()
top_rate = country_detection_counts.loc[top_country, 'detection_rate']

print("Source country with highest detection rate for Email Spam attacks targeting Firewall systems (payload > 4000 bytes):")
print(f"Country: {top_country}")
print(f"Detection Rate: {top_rate:.2%}")


Source country with highest detection rate for Email Spam attacks targeting Firewall systems (payload > 4000 bytes):
Country: Brazil
Detection Rate: 88.89%


In [ ]:
# Define destination port ranges as tuples (start, end)
port_ranges = {
    '0-10000': (0, 10000),
    '10000-20000': (10000, 20000),
    '20000-30000': (20000, 30000),
    '40000-50000': (40000, 50000),
}

# Step 1: Filter for detected Watering Hole attacks from Japan
filtered_df = df[
    (df['Detection Label'].str.lower() == 'detected') &
    (df['Attack Type'].str.contains('watering hole', case=False, na=False)) &
    (df['Source Country'].str.lower() == 'japan')
].copy()

# Step 2: Function to categorize destination ports into ranges
def categorize_port(port):
    for label, (start, end) in port_ranges.items():
        if start <= port < end:
            return label
    return 'Other'

# Apply categorization (make sure Destination Port is numeric)
filtered_df['Destination Port'] = pd.to_numeric(filtered_df['Destination Port'], errors='coerce')
filtered_df = filtered_df.dropna(subset=['Destination Port'])

filtered_df['Port Range'] = filtered_df['Destination Port'].apply(categorize_port)

# Step 3: Count frequencies of each port range
port_range_counts = (
    filtered_df
    .groupby('Port Range')
    .size()
    .reset_index(name='Count')
    .sort_values(by='Count', ascending=False)
)

print("Detected Watering Hole attacks from Japan by destination port range:")
print(port_range_counts)

# Step 4: Identify the port range with the highest count (ignore 'Other' if present)
top_port_range = port_range_counts[port_range_counts['Port Range'] != 'Other'].iloc[0]

print(f"\nPort range with highest frequency: {top_port_range['Port Range']} ({top_port_range['Count']} attacks)")


Detected Watering Hole attacks from Japan by destination port range:
    Port Range  Count
4        Other     98
0      0-10000     53
3  40000-50000     41
1  10000-20000     32
2  20000-30000     28

Port range with highest frequency: 0-10000 (53 attacks)


In [ ]:
# Define port ranges as tuples: (start, end)
port_ranges = {
    '0-9999': (0, 9999),
    '10000-19999': (10000, 19999),
    '20000-29999': (20000, 29999),
    '30000-39999': (30000, 39999),
    '40000-49999': (40000, 49999),
    '50000-65535': (50000, 65535)
}

# Filter for detected Watering Hole Attack from Japan
filtered_df = df[
    (df['Detection Label'].str.lower() == 'detected') &
    (df['Attack Type'].str.contains('watering hole', case=False, na=False)) &
    (df['Source Country'].str.lower() == 'japan')
].copy()

# Assign port range label to each row based on destination port
def assign_port_range(port):
    for label, (start, end) in port_ranges.items():
        if start <= port <= end:
            return label
    return 'Unknown'

filtered_df['Port Range'] = filtered_df['Destination Port'].apply(assign_port_range)

# Count frequency of attacks per port range
port_range_counts = (
    filtered_df
    .groupby('Port Range')
    .size()
    .reset_index(name='Attack Count')
    .sort_values(by='Attack Count', ascending=False)
)

# Output the result
print("Frequency of detected Watering Hole Attacks from Japan by destination port range:")
print(port_range_counts)

if not port_range_counts.empty:
    top_range = port_range_counts.iloc[0]
    print(f"\nPort range with highest frequency: {top_range['Port Range']} ({top_range['Attack Count']} attacks)")
else:
    print("No matching detected Watering Hole Attacks from Japan found.")


Frequency of detected Watering Hole Attacks from Japan by destination port range:
    Port Range  Attack Count
5  50000-65535            59
0       0-9999            53
4  40000-49999            41
3  30000-39999            39
1  10000-19999            32
2  20000-29999            28

Port range with highest frequency: 50000-65535 (59 attacks)


In [ ]:
import pandas as pd
import numpy as np

# Ensure Timestamp is datetime
df['Timestamp'] = pd.to_datetime(df['Timestamp'], errors='coerce')

# Filter for detected DDoS attacks on Firewall systems in 2024
filtered_df = df[
    (df['Timestamp'].dt.year == 2024) &
    (df['Detection Label'].str.lower() == 'detected') &
    (df['Attack Type'].str.contains('distributed denial of service|ddos', case=False, na=False)) &
    (df['Affected System'].str.contains('firewall', case=False, na=False))
].copy()

# Calculate median payload size across these filtered attacks
median_payload = filtered_df['Payload Size (bytes)'].median()

# Keep only attacks with payload size above median
filtered_df = filtered_df[filtered_df['Payload Size (bytes)'] > median_payload]

# Define a function to map month to quarter label
def month_to_quarter(month):
    if month in [1, 2, 3]:
        return 'Q1 (Jan-Mar)'
    elif month in [4, 5, 6]:
        return 'Q2 (Apr-Jun)'
    elif month in [7, 8, 9]:
        return 'Q3 (Jul-Sep)'
    else:
        return 'Q4 (Oct-Dec)'

# Create a 'Quarter' column
filtered_df['Quarter'] = filtered_df['Timestamp'].dt.month.apply(month_to_quarter)

# Group by Quarter and count attacks
quarter_counts = (
    filtered_df
    .groupby('Quarter')
    .size()
    .reset_index(name='Attack Count')
    .sort_values(by='Attack Count', ascending=False)
)

# Get the quarter with the highest number of attacks
top_quarter = quarter_counts.iloc[0]

print("Quarter in 2024 when Firewall systems faced the most detected DDoS attacks with payload size above median:")
print(f"{top_quarter['Quarter']} with {top_quarter['Attack Count']} attacks")


Quarter in 2024 when Firewall systems faced the most detected DDoS attacks with payload size above median:
Q2 (Apr-Jun) with 4 attacks
